<h1 style="text-decoration:underline">GSoC Project Proposal</h1>
<h1 style="text-decoration:underline">Add Variational Inference Interface to PyMC4</h1>

## Description
`Variational Inference` is a powerful algorithm that turns the task of computing the posterior(p(z|x)) into an optimization problem.  This project is about implementing two inference algorithms Mean Field ADVI and Full Rank ADVI based on [ADVI](https://arxiv.org/abs/1603.00788) paper in PyMC4. Mean Field ADVI posits a spherical Gaussian family and Full Rank ADVI posits a Multivariate Gaussian family to minimize KL divergence. The implementation will use tf and tfp libraries. 

## Interface Design

In [ ]:
# Base class for Approximation
class Approximation:
    
    # Defining parameters
    def __init__(self, model, size, random_seed=None):
        self.model = model
        self.size = size
        self.random_seed = random_seed
        # Handle initialization of mu and std
        
    @property
    def mean(self):
        return self.mu

    @property
    def std(self):
        return self.std

    # For Naive Monte Carlo
    def random(self):
        g = tf.random.Generator.from_seed(self.random_seed)
        n = g.normal(shape=some_shape)
        return self.std*n + self.mean

In [ ]:
class MeanField(Approximation):
    
    def __init__(self, model, size, random_seed):
        super().__init__(model, size, random_seed)
        
    def cov(self):
        sq = tf.math.square(self.std)
        return tf.linalg.diag_part(sq)


class FullRank(Approximation):
    
    def __init__(self, model, size, random_seed):
        super().__init__(model, size, random_seed)
    
    def L(self):
        n = self.size
        entries = n*(n+1)//2
        L = np.zeros([n, n], dtype=int)
        L[np.tril_indices(n)] = np.arange(entries)
        L[np.tril_indices(n)[::-1]] = np.arange(entries)
        return L
        
    def cov(self):
        L = self.L
        return tf.linalg.matmul(L, tf.transpose(L))

In [ ]:
def fit(model, n=10000, random_seed=None, method='MeanFieldADVI'):
    
    # Transform the model into an unconstrained space
    _, state = pm.evaluate_model_transformed(model)
    logpt = state.collect_log_prob()
        
    # Collect the free random variables
    untransformed = state.untransformed_values
    free_RVs = untransformed.update(state.transformed_values)
    
    # Not sure about the use of local random variables
    size = 0
    for name, dist in free_RVs.items():
        size += int(np.prod(dist.event_shape))
    
    approx = None
    if method == "MeanFieldADVI":
        approx = MeanField(model, size, random_seed)
    else:
        approx = FullRank(model, size, random_seed)

    # Create variational gradient tensor
    q = approx.random()
    elbo = q + tf.reduce_sum(approx.std) + 0.5*size*(1 + tf.math.log(2.0*np.pi))
    
    # Set up optimizer
    
    # Draw samples from variational posterior

    # TODO: Plot the trace using ArviZ

## Key Challenges

1. **theano.clone equivalent for Tf2**: For drawing samples from posterior, we need to replace some nodes of the graph with variational distributions. PyMC3 achieves this by using `theano.clone`. Tensorflow v1 implements some similar functions in the `contrib` module - [tf.contrib.copy_graph](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/contrib/copy_graph). But the tf.contrib has been deprecated in TF2. Support of symbolic graph manipulations has also been removed. 
 - `tf.keras.backend` can be used for symbolic graphs. 
 - `tf-nightly` provides [tf.raw_ops.Copy](https://www.tensorflow.org/api_docs/python/tf/raw_ops/Copy) for recursive deep copying of tensor. Maybe it is possible to implement a feature similar to `theano.clone` by looking upon implementations of both `copy_graph`(in tf.v1) and `Copy`(in tf.nightly).


2. Provide an interface for flattened view of variables. PyMC3 does this using `theano.ravel()`. Maybe it can be accomplished by `tf.reshape`.

3. Design an interface to account for minibatches. 

4. Figure out a way to optimize ELBO. `tf.keras.optimizers` is an option or provide an iterative optimization algorithm.

5. Handle the initialization of means and stds for MeanField and Full Rank ADVI.

6. Provide a reusable interface to implement more inference algorithms (SVGD, ASVGD).

7. Progress Bar ?

## Project Timeline
 - Pre GSoC Period: March 10 - March 20 <br/>
Figure out the solutions of Challenges described above. I will make design less and less abstract as the time progresses and submit the proposal.


 - Student Application and Review Application Period: March 20 - April 27 <br />
I will get myself more familiar with codebase of PyMC3, PyMC4 and begin implementing a few functions.


 - Community Bonding Period: April 27 - May 18 <br/> 
I will finalize implementation design for both algorithms by seeking reviews from mentors and community and also referring to OPVI and ADVI paper.


 - Week 1 - 2: May 18 - May 31 <br/>
The actual coding begins. By this time, I will have a fair idea of interface design. I will begin implementing base Approximation class.


 - Week 3 - 4: June 1 - June 14 <br/>
During this interval, I will implement utility functions (theano.clone and flatten). For every functionality, I will write test cases. Also I will create a progress report for phase 1 evaluations.


 - Evaluation Phase 1: June 15 - June 19 <br/>
At this time, my focus will be on writing visual illustrations/notebooks for use cases of these base classes.


 - Week 5 - 6: June 20 - July 3 <br/>
Having implemented base classes, I will write Mean Field Approximation class and pm.fit function to make inference. 


 - Week 7: July 4 - July 12 <br/>
I am not sure if I will be able to complete the implementation of Mean Field and fit function. I will also look for handling inference for mini batches. Designing progress reports.


 - Evaluation Phase 2: July 13 - July 17 <br/>
Again, writing notebooks/blogs to explain good use cases.


 - Week 8 - 9: July 18 - July 31 <br />
Implementation of FullRank Approximation class. Alter the pm.fit function to also account for Full Rank advi option.


 - Week 10: August 1 - August 9 <br />
Account for mini batches in Full Rank ADVI. Tests in Pytest. Writing illustrations/blogs.


 - Final Submission of Code: August 10


 - Post GSoC Period: August 11 onwards <br/>
I will learn and implement SVGD, ASVGD inference algorithms and become a permanent contributor to the organisation. 

## Contributing to PyMC4
1. Pull Request [#220](https://github.com/pymc-devs/pymc4/pull/220) (Merged): Add AutoRegressive distribution - <br/>
This PR added Auto Regressive distribution by wrapping `sts.AutoRegressive` Model. The main task was to call `make_state_space_model` method with suitable arguments to capture the underlying the `tfd.LinearGaussianStateSpaceModel`. It took a lot of debugging to make this AR class compatible with PyMC4.

2. Pull Request [#215](https://github.com/pymc-devs/pymc4/pull/215) (Merged): Add default transform(sigmoid) for Unit Continuous Distribution - <br/>
This PR added sigmoid transform to Unit Continuous Distribution. To make the default transform compatible with PyMC4, I also added Sigmoid transform that used `tfb.Sigmoid` bijector.

3. Pull Request [#212](https://github.com/pymc-devs/pymc4/pull/212) (Merged): Update design_guide notebook - <br/>
This small PR fixed typos and variable names in `pymc4_design_guide.ipynb`.

4. Issue [#211](https://github.com/pymc-devs/pymc4/issues/211) (Closed): Installation issues
I encountered installation issues while setting up the working environment using pip. So, I created the issue and Luciano Paz helped me out with other ways of installing PyMC4.

## Personal Projects
1. Send to S3 - [Github](https://github.com/Sayam753/SendToS3) <br/>
This python project sends backup files to AWS S3 bucket using Boto3. Searching for files is done by regex and results of logs are sent to email using smtplib.

2. Osint-Spy - [Github](https://github.com/Sayam753/OSINT-SPY) <br/>
This Python project performs Osint scan on email, domain, ip, organization, etc.
This information can be used by Data Miners or Penetration Testers in order to find deep information about their target.

3. Turbofan Degradation - [Colab](https://colab.research.google.com/drive/1sCZcJSmRarYbQKDYeaqiLnzXyzFolRC0) <br/>
Implemented a Deep learning based Encoder-Decoder model ([paper](https://www.researchgate.net/publication/336150924_A_Novel_Deep_Learning-Based_Encoder-Decoder_Model_for_Remaining_Useful_Life_Prediction)) for analysing the turbofan degradation dataset provided by NASA.

4. Neural Network from Scratch - [Colab](https://colab.research.google.com/drive/1iU38tTeEvUI_sjt6vVAuhedMWOPUdr5E) <br/>
Implemented a deep neural network from scratch in numpy with custom hyperparameters.

## Basic/Contact Information

 - Time Zone: UTC+05:30
 - Github Handle: [Sayam753](https://github.com/Sayam753)
 - Resume: [Google drive link](https://drive.google.com/file/d/1mrNC3qtieWKH1i2mhqH6xiFCt-EwGJ0b/view?usp=sharing), [Github link](https://github.com/Sayam753/Resume)
 - Contact details: [Gmail](sayamkumar049@gmail.com), [Yahoo](sayamkumar753@yahoo.in), [LinkedIn](https://www.linkedin.com/in/sayam049/), [Twitter](https://twitter.com/sayamkumar753), +91 9815247310 (Mobile)

### Personal Info
I am Sayam Kumar from Indian Institute of Information Technology Sri City, India. I am a second year Undergraduate pursuing a Bachelor's in Computer Science Engineering. I mostly code in Python. I am interested to work on the project to expand my knowledge in Machine Learning and Bayesian Statistics. With my continuous efforts to learn and know more about Bayesian Statistics, I believe I will be able to complete the project. Also this is my first time participating in GSoC. 

### Commitments
As I have no other projects/internships planned for summers, I can spend 40~50 hours per week or more if required working on the project. Along the way, I will design progress reports and extensive documentation of the implementation of various classes. This will help in submitting reports to mentor and Google at evaluation time.


## References and Useful Papers
 - [Automatic Differentiation Variational Inference](https://arxiv.org/pdf/1603.00788.pdf). Kucukelbir, A., Tran, D., Ranganath, R., Gelman, A., and Blei, D. M. (2016).
 - [Automatic Variational Inference in Stan](https://arxiv.org/abs/1506.03431). Kucukelbir, A., Ranganath, R., Gelman, A., & Blei, D. (2015).
 - [Operator Variational Inference](https://arxiv.org/abs/1610.09033). Rajesh Ranganath, Jaan Altosaar, Dustin Tran, David M. Blei (2016).
 - [Auto-Encoding Variational Bayes](https://arxiv.org/abs/1312.6114). Kingma, D. P., & Welling, M. (2014).